# Main simulation code - same time

<hr style="clear:both">
This notebook runs all the simulation. The conditions defining this simulation should be defined in the notebook called "classes.ipynb". Only the objectif function can be change in this notebook.

**Authors:** [Lynn Fayed](https://people.epfl.ch/lynn.fayed), [Lorenzo Ballinari](https://people.epfl.ch/lorenzo.ballinari), [Paulo Alexandre Ribeiro de Carvalho](https://people.epfl.ch/paulo.ribeirodecarvalho)

<hr style="clear:both">

## Librairies

In [1]:
import numpy as np
import pdb
import cProfile
import timeit
import networkx as nx
from scipy.optimize import linear_sum_assignment
import time
import random
import copy
from copy import deepcopy
import pickle
from ipywidgets import IntProgress
from IPython.display import display
import time
import os

## Import datas

In [2]:
# import network data

data_file = open(r"data.pkl", "rb")
f = pickle.load(data_file)
arrival_batches, alldists, allpaths = f[0], f[1], f[2]

In [3]:
# import notebooks

%run func_same_time.ipynb
%run classes.ipynb
np.random.seed(0)

## Initialize 

In [4]:
# Initialize input variables

input_variables = Input_Variables()
private_vehicles = Private_Vehicles()
ride_hailing_vehicles = Ride_Hailing_Vehicles(alldists, input_variables.fleet_size)
traveling_passengers = Traveling_Passengers()
waiting_requests = Waiting_Requests()

# Initialize arrivals

arrivals = arrival_batches[input_variables.batch_number]
r = np.random.rand(len(arrivals[0])) # donne une "liste" (array de 1Xlen(arrivals[0])) de chiffre aléatoire entre 0 et 1.
r = r < input_variables.fraction_of_private_vehicles # retourne une liste avec des True et False ou la conditions est respectée ou pas. 
private_vehicle_arrivals = arrivals[:,r]
request_arrivals = arrivals[:,r == False]

# Initialize output variables

accumulation, velocity = [], []
abandonment, total_arrived_requests = 0, 0
accumulation.append(input_variables.fleet_size)
velocity.append(compute_speed(accumulation[0]))

## Conditions

Give the paramaters choosen for the current simulation

In [5]:
print(f'fleet size : {input_variables.fleet_size}[veh]\nmax waiting time : {input_variables.maximum_waiting_time*60}[min]\nmax detour time : {input_variables.maximum_detour_time*60}[min]\nsharing percentage : {input_variables.sharing_fraction*100}[%]')

fleet size : 500[veh]
max waiting time : 5.0[min]
max detour time : 15.0[min]
sharing percentage : 80.0[%]


## Simulation

In [6]:
######################################## simulation code ########################################
# Let's start the chrono to know the duration of the simulation
t1 = time.time()

# Discretize the time in steps
Time = np.arange(input_variables.time_step, input_variables.duration_of_simulation, input_variables.time_step)

# Initialize the list to stock the performance variable
Occupancy_perf = np.ones((3,len(Time)))
Abandonment_perf = np.array([])
already_analyze = []
Waiting_time_perf = np.array([])
Detour_time_perf = []

# Initialize the first step
traveling_passengers_old = deepcopy(traveling_passengers)
requests_are_sharing = []

# Initialize the history of ID that will share and with wich scenario
history_sharing = np.zeros((1,3))

# Progression bar
print("Progress bar of the simulation")
progression = 0
f = IntProgress(min=0, max=len(Time)) # instantiate the bar
display(f) # display the bar

for time in Time:
    # Update progress bar
    f.value += 1 # signal to increment the progress bar
    progression += 1
    print(f'{(progression/len(Time))*100:.2f}[%]', end='\r')
    
    # Update network
    traveled_distance = input_variables.time_step * velocity[-1]
    
    for vehicle_index in range(input_variables.fleet_size):
        update_remaining_distance(ride_hailing_vehicles, vehicle_index, traveled_distance )
        update_network(ride_hailing_vehicles, waiting_requests, traveling_passengers, vehicle_index)
    
    if np.any(private_vehicles.remaining_distance):
        private_vehicles.remaining_distance -=  traveled_distance
    
    if len(traveling_passengers.traveled_time):
        traveling_passengers.traveled_time +=  traveled_distance/velocity[-1]
        
    # Update arrivals
    new_private_vehicle_arrivals = private_vehicle_arrivals[:, (private_vehicle_arrivals[0]>=time - input_variables.time_step) & (private_vehicle_arrivals[0]<time)]
    new_request_arrivals = request_arrivals[:, ((request_arrivals[0]>=time - input_variables.time_step) & (request_arrivals[0]<time))]
    compute_private_vehicle_arrivals(private_vehicles, new_private_vehicle_arrivals)
    compute_request_arrivals(waiting_requests, input_variables.sharing_fraction, new_request_arrivals)
    total_arrived_requests  += len(new_request_arrivals)
    
    # Update abandonments
    exiting_bool = ((time - waiting_requests.request_arrival_time) > input_variables.waiting_tolerance) * (waiting_requests.assigned_driver == -1) #vecteur remplit de 1 (request à abandonner) et de 0 (si la request peut encore être traité) 
    abandon_requests(waiting_requests, private_vehicles, exiting_bool) #Remove abandoning vehicles from list of passengers and add them to pvs
    abandonment += sum(exiting_bool) #on somme le vecteur remplit de 1 et 0 pour savoir combien de request ont été abandonné
    
    # Update number of private vehicles in the system
    complete_private_vehicle_trips(private_vehicles)
    
    # Keep an eye on evolution of some performance metric
    occupancy_perf(Occupancy_perf, time, ride_hailing_vehicles, Time)
    Abandonment_perf = abandonment_perf(Abandonment_perf, abandonment)
    Waiting_time_perf = waiting_time_perf(Waiting_time_perf, traveling_passengers, time, already_analyze)
            
    # Match vehicles to requests
    if np.round(time/input_variables.time_step, 5)%input_variables.batching_time_factor==0: #check si le rapport de "time" sur le "time step" (arrondie au 5) est un multiple de "batching_time_factor"

        # The class waiting_requests already sort the request by the time they arrived.
        index_unassigned_request = (waiting_requests.assigned_driver == -1).nonzero()[0] # We have now the index of all unassigned request that we must try to fit with a sharing car. 
        index_unassigned_request_ready_to_share = np.array([])
        
        # We only keep requests that are ready to share the ride
        for i in index_unassigned_request:
            if waiting_requests.willingness_to_share[i] == 1:
                index_unassigned_request_ready_to_share = np.append(index_unassigned_request_ready_to_share, int(i))           
    
        # Occupancy
        index_vehicles_available = occupancy(ride_hailing_vehicles)
        
        # List of scenarios
        scenarios = [0,1,2,3,4,5,6]
        
        # Initialize cost matrix shape(#veh,#request)
        cost_matrix = np.zeros((len(index_vehicles_available),len(index_unassigned_request)))
        scenario_matching = np.zeros((len(index_vehicles_available),len(index_unassigned_request)))
        
        # Begin the constraint control
        for request_ID in index_unassigned_request:
            for vehicle_ID in index_vehicles_available:
                
                request_ID = int(request_ID)
                vehicle_ID = int(vehicle_ID)
                
                # Initialize the list to stock the metrics
                metrics = []
                metrics_feasibilty = []
                                     
                for scenario in scenarios:
                    
                    # Waiting time
                    check_w, waiting_time_1, waiting_time_2 = check_waiting_time(request_ID, vehicle_ID, scenario, 
                                                                                 ride_hailing_vehicles, waiting_requests, 
                                                                                 traveling_passengers, input_variables, time)
        
                    # Detour time
                    check_t, detour_time_1, detour_time_2 = check_detour_time(request_ID, vehicle_ID, scenario, 
                                                                              ride_hailing_vehicles, waiting_requests, 
                                                                              traveling_passengers, input_variables, 
                                                                              time, velocity[-1], check_w)
                    
                    # Stock metrics results
                    metrics.append(avg(detour_time_1, detour_time_2)) #choose here the metric to optimize
                    metrics_feasibilty.append(check_t) # where there is a 1 in this list means that both constraint are okay.
                        
                # Update cost matrix
                cost_matrix, scenario_matching = update_matrix(request_ID, vehicle_ID, cost_matrix, scenario_matching, metrics, 
                                                               metrics_feasibilty, 
                                                               index_vehicles_available, 
                                                               index_unassigned_request)
                

        history_sharing = match_sharing(waiting_requests, ride_hailing_vehicles, cost_matrix, scenario_matching, 
                                        Detour_time_perf, traveling_passengers, input_variables, time, velocity[-1], 
                                        1, history_sharing)
    
    # Keep an eye on detour times
    traveling_passengers_old = detour_time_perf(traveling_passengers, traveling_passengers_old, velocity[-1], 
                                                requests_are_sharing, Detour_time_perf)      

    # Update accumulation and speed
    accumulation.append(input_variables.fleet_size + len(private_vehicles.remaining_distance))
    velocity.append(compute_speed(accumulation[-1]))
    
output = {'abandonment': abandonment}

# Stop the chrono and give the all duration for this simulation
del time # because an other variable has been call like this, so we need to delete it...
import time # ... and then re-import time. 
t2 = time.time()
print(f'The current simulation took {int((t2-t1)/60)}[min] and {int((t2-t1)-(int((t2-t1)/60)*60))} [s] to finsh.')

Progress bar of the simulation


IntProgress(value=0, max=2159)

/tmp/ipykernel_558/4020577522.py:6: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if traveling_passengers.request_ID[index] in already_analyze:


The current simulation took 11[min] and 9 [s] to finsh.


## Results

In [7]:
# Convert the list in array, this way all performance variables are the same class
Detour_time_perf=np.asarray(Detour_time_perf)

# Print all 4 performances variables & history of sharings
# print("position de la dernière mise à jour : ",int((time/Time[-1])*len(Time)),"\n")
# print("occupancy performance : ",Occupancy_perf.shape,type(Occupancy_perf),"\n",Occupancy_perf, "\n")
# print("abandonment performance : ",Abandonment_perf.shape,type(Abandonment_perf),"\n",Abandonment_perf,"\n")
# print("waiting time performance : ",Waiting_time_perf.shape,type(Waiting_time_perf),"\n",Waiting_time_perf,"\n")
# print("detour performance : ",Detour_time_perf.shape,type(Detour_time_perf),"\n",Detour_time_perf, "\n")
# print("history of sharing : ",history_sharing.shape,type(history_sharing),"\n",history_sharing, "\n")

## Save results

Be sure to have all the path created in your computer.

In [8]:
# Give a feedback of the parameters for the current simulation
print(f'fleet size : {input_variables.fleet_size}[veh]\nmax waiting time : {input_variables.maximum_waiting_time*60}[min]\nmax detour time : {input_variables.maximum_detour_time*60}[min]\nsharing percentage : {input_variables.sharing_fraction*100}[%]')

fleet size : 500[veh]
max waiting time : 5.0[min]
max detour time : 15.0[min]
sharing percentage : 80.0[%]


In [9]:
number = '3_reduce'

# Save all 4 performances variables
np.save(f'Results_same_time/condition_{number}/Occupancy_perf_condition_{number}_s_t.npy', Occupancy_perf)
np.save(f'Results_same_time/condition_{number}/Abandonment_perf_condition_{number}_s_t.npy', Abandonment_perf)
np.save(f'Results_same_time/condition_{number}/Waiting_time_perf_condition_{number}_s_t.npy', Waiting_time_perf)
np.save(f'Results_same_time/condition_{number}/Detour_time_perf_condition_{number}_s_t.npy', Detour_time_perf)

# Save the history of every request that share a ride
np.save(f'Results_same_time/condition_{number}/history_sharing_condition_{number}_s_t.npy', history_sharing)

In [10]:
# Make a sound when simulation is finish
#os.system('say "votre simulation est terminée !"')